In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from scipy.special import expit
from scipy.optimize import fmin, minimize
import glob as glob
import matplotlib
matplotlib.style.use('ggplot')

# .py
%load_ext autoreload
%autoreload 2
import weather_alphas, weather_costs, model, model_01, validate, prediction

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


running model
getting cost from nweek ahead
getting bounded alphas ...
done getting alphas ...
running model


In [2]:
def split_data(data):
    train_mask = (data.index < "2011")
    train = data[train_mask]
    test_mask = (data.index > "2011")
    test = data[test_mask]
    return train, test

In [3]:
all_weather = pd.read_csv("weather01-16.csv")

In [4]:
cm_weather = all_weather[all_weather.stn_name == "Chiang Mai"]
cm_weather.index = pd.DatetimeIndex(cm_weather.date)
mask = (cm_weather.index > "2003") & (cm_weather.index < "2016")
cm_weather = cm_weather[mask]

provinces_df = pd.read_csv("all-dengues.csv")
provinces_df.index = pd.DatetimeIndex(provinces_df.date)
provinces_df = provinces_df.drop(['date','date.1'],axis=1)
cm_dengues = provinces_df[provinces_df['จังหวัด'] == 'ChiangMai'].resample('W').size()
cm_dengues_df = pd.DataFrame(cm_dengues,columns=['cases'])

In [5]:
cm_avg_weather = cm_weather[['avgrh','dday','meantemp']].resample('W').mean()
cm_avg_weather['rain'] = cm_weather[['rain']].resample('W').sum() # cumulative rainfall
cm_dengues_weather = pd.concat([cm_avg_weather,cm_dengues_df[:-52]],axis=1)

cm_dengues_weather_split = split_data(cm_dengues_weather)
cm_dengues_train, cm_dengues_test = cm_dengues_weather_split[0], cm_dengues_weather_split[1]

In [6]:
def make_line(start_week,real,predictions,title,x_axis,y_axis,
              real_legend='Real',predict_legend='Prediction'):
    plt.figure(figsize=(12,6))
    plt.plot(range(len(real)),real,'-r',label=real_legend)
    plt.plot(range(start_week,len(predictions)+start_week),predictions,'-k',label=predict_legend,alpha=0.4)
    plt.axvline(start_week)
    plt.text((start_week*2)+10,300,'Prediction Start',horizontalalignment='center',verticalalignment='center')
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.legend()
    plt.title(title)
    
def plot_all_predictions(LAG, real, province, nweeks_to_predict, all_predictions):
    
    plt.figure(figsize=(13,6))
    plt.plot(range(len(real)),real,label='real')
    graph_title = "%s's Dengue Prediction with Mean Temperature"%province
    plt.title(graph_title)
    for i in range(len(all_predictions)):
        cur_week_guess = nweeks_to_predict[i]
        start_predict_week = LAG+(cur_week_guess-1)
        predicted_cases = all_predictions[i]
        x_weeks = range(start_predict_week,len(predicted_cases)+start_predict_week)
        line_label = "%d week guess"%cur_week_guess
        plt.plot(x_weeks,predicted_cases,label=line_label,alpha=0.8)
    
    seperate_train_test(cm_dengues_df.cases,cm_dengues_train)
    plt.axvline(LAG)
    plt.text((LAG*2)+10,300,'Prediction Start',horizontalalignment='center',verticalalignment='center')
    plt.xlabel("# week starting 2003")
    plt.ylabel("Dengue cases")
    plt.legend()
    plt.savefig("16-lags-16-weeks-9-temp-wks-chiangmai-00.png")
    
def seperate_train_test(all_data,train_data):
    plt.axvspan(0, len(train_data), color='blue', alpha=0.1)
    plt.text((len(train_data)/2)-25,700,'Train Period',size=16)
    plt.axvline(len(train_data)+1,linewidth=0.5, color='green')
    plt.text(len(train_data)+20,700,'Test Period',size=16)
    plt.axvspan(len(train_data)+1, len(all_data), color='green', alpha=0.1)

In [7]:
import multiprocessing as mp
from multiprocessing import Process, Queue

# Define an output queue
output = Queue()

# def validate

# def compute_ret(temp):
#     for i in range(int(1e5)): pass
#     return temp/2.


ws_csv = np.array(pd.read_csv("best-for-22-lags-ws.csv").T)[0]
    
def validation(LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, train, real, output):
                                    # get_alphas takes 4 args, if theres no input csv
    alphas = weather_alphas.get_alphas(LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, train, ws_csv)
                                    # get_predictions takes 6 args
    predictions_to_validate = prediction.get_predictions(LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, real, alphas.x, "CM")
                                    # get_validations takes 3 argsg
    validated_result = validate.get_validations(LAG, predictions_to_validate, real)
    output.put((LAG,validated_result,alphas.x)) # add in ws as well alphas.x


In [9]:
%%time
# Setup a list of processes that we want to run
train = cm_dengues_train
real = cm_dengues_weather
processes = [Process(target=validation, args=(weeks, 9, 10, train, real, output)) for weeks in range(19,29)]

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

43
bnds 43
ws_csv
44
prev_ws 43
45
bnds 44
ws_csv
bnds 45
46
ws_csv
prev_ws 44
bnds 46
prev_ws 45
ws_csv
47
prev_ws 46
bnds 47
ws_csv
48
prev_ws 47
bnds 48
49
bnds 49
ws_csv
ws_csv
prev_ws 49
51
prev_ws 48
bnds 51
52
ws_csv
bnds 52
50
prev_ws 51
ws_csv
bnds 50
ws_csv
prev_ws 52
prev_ws 50
[(28, [238.68440771959951, 333.98675844456238, 628.97375259711259, 1088.5052949338437, 1494.2309652728031], array([  1.11179702e+00,   1.91072806e-10,   2.44118551e-10,
         0.00000000e+00,   2.09882553e-10,   9.88341128e-03,
         0.00000000e+00,   0.00000000e+00,   5.87555908e-05,
         5.45288352e-03,   3.00065362e-02,   1.39235869e-01,
         2.28433225e-01,   1.59313504e-02,   2.31287849e-04,
         5.14480842e-05,   1.62542679e-05,   3.81501234e-07,
         1.75792159e-07,   8.87055488e-08,   3.19903701e-09,
         0.00000000e+00,   0.00000000e+00,   2.24686207e-09,
         0.00000000e+00,   3.08344958e-08,   1.11972195e-01,
         4.63928140e-01,   4.27883577e-01,  -7.748372

In [21]:
def get_best(validations):
    idx = None
    best_so_far = validations[0][1][-1]
    print best_so_far
    for i in range(len(validations)):
        cur_validation = validations[i][1]
#         print cur_validation
        if cur_validation[0] <= best_so_far:
            best_so_far = cur_validation[0]
            idx = i
    return validations[idx]
        
    
best_param = get_best(results)

1658.87429138


In [22]:
best_param

(22,
 [168.91992707941367,
  294.43310789284749,
  581.26512237331235,
  1094.9170235268389,
  1583.9765963195935],
 array([  1.06285782e+00,   3.18700091e-09,   7.32517050e-02,
          2.47404558e-05,   1.80339246e-04,   4.42235741e-03,
          8.05425584e-03,   1.40797148e-02,   3.70112369e-03,
          5.80911759e-02,   2.59760472e-03,   8.49970251e-04,
          1.75154465e-04,   7.70080825e-05,   2.34779063e-05,
          9.25569626e-07,   2.78796997e-07,   8.92087310e-08,
          5.36438747e-08,   1.10259113e-08,   3.72087335e-08,
          1.67251300e-08,   4.45071577e-01,   1.78414402e+00,
          1.14920133e+01,   5.28767936e-01,   2.91518901e-01,
          1.39171499e-01,  -1.12398539e-01,  -1.24287564e-01,
         -1.99536889e-01,   3.83116461e-02,  -4.25686363e-02,
         -1.85499470e-01,  -3.30833805e-01,   5.49950430e-01,
          6.64274568e-03,   2.89012569e-03,  -6.17473643e-06,
          7.22101455e-04,   5.23957335e-03,  -5.45793794e-03,
          4.7101

In [23]:
# pd.DataFrame(best_param[2]).to_csv("best-for-22-lags-ws.csv",index=False)
# np.array(pd.read_csv("best-for-21-lags-ws.csv").T)[0]

In [147]:
# # pd.DataFrame(best_param[2]).to_csv("best-for-18-lags-ws.csv",index=False)

# lg = [16,17,18,20,22]
# a = np.array(pd.read_csv("best-for-18-lags-ws.csv").T)[0]

# # for i in lg:
# #     if i == len(a):
# #         arr = a
# #         print len(arr)
# #     elif i < len(a):
# #         arr = a[:-(18-i)]
# #         print len(arr)
# #     else:
# #         mean_val = 1/float(i)
# #         arr = np.append(a,[mean_val]*(i-len(a)))
# #         print arr
        
# def ws_helper(LAG, ws, 18):
#     n_ws = len(ws)
#     if LAG == n_ws:
#         print "eq"
#         arr = ws[:LAG+1]
#     elif LAG < n_ws:
#         print "les"
#         arr = ws[:-(n_ws - LAG + 1)]
#     else:
#         print "else"
#         tail = (LAG - n_ws) - 1
#         mean_val = 1/(float(LAG)*tail)
#         arr = np.append(ws,[mean_val]*tail)
#     return arr

# len(ws_helper(20,a))

les


19

In [69]:
%%time
[compute_ret(temp.get_alphas(i,9,10)) for i in range(16,26)]

NameError: name 'compute_ret' is not defined

In [10]:
print "im done"

im done
